In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


**FastText activity**

In [ ]:
! pip install fasttext

     |████████████████████████████████| 71kB 3.8MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3092386 sha256=6ba2a98f8577563b14d20b7bfbb4c68c5cc3f617f91c088894ef3bc6ca4f481d
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
import string
import nltk
nltk.download('stopwords')
nltk.download('words')
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import csv
import os
from functools import partial
import torch
import fasttext
import sys
import os
import pandas as pd
import numpy as np

from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

WORDS = set(nltk.corpus.words.words())

ignore = ['nan']

print(len(WORDS))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
235892


In [3]:
datafile = "/content/gdrive/MyDrive/NLP_Final/data/kaggle_RedditSarcasm/train-balanced-sarcasm.csv"

In [4]:
# read the data
import pandas as pd
df = pd.read_csv(datafile)

In [8]:
# make lowercase 
df['comment'] = df['comment'].str.lower()

In [7]:
# shuffle and show the first few posts
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,1,Probably because employers spend so much money...,DuckSmash,news,-1,-1,-1,2016-10,2016-10-06 17:28:55,Working class white men have lower incomes tha...
1,0,SHRC never was a good team they had shitty dom...,LucianLvL2PowerSpike,leagueoflegends,2,2,0,2015-08,2015-08-10 12:20:50,"King (Insec, Zero, Namei, Corn, etc) got releg..."
2,1,"but... but.. you guys, it takes a village!",eli5ask,pics,1,1,0,2015-10,2015-10-29 19:21:49,So ... beggars can be choosers?
3,1,"Yes, internet drama is more important than har...",Draumbr,Vive,-5,-5,0,2015-09,2015-09-19 22:26:17,I'm glad I saw this. My wife an I are due to s...
4,1,"Yes, all teenagers lack any wisdom whatsoever.",dracdliw,funny,1,1,0,2012-06,2012-06-19 03:26:53,Cancerish. Cancerish.... Saying teenagers can ...


In [ ]:
# do preprocessing and fasttext data utility

"""
  Helper to get the data into the format expected by fast text

  df- dataframe of reddit posts as shown above
  fname - the name of the file to save to should be .txt
"""
def makeFastTextData(df, fname=None):
  assert fname[-4:] == ".txt", "file must have .txt extension"
  with open(fname, 'w') as f:
    for et, el in zip(df['comment'], df['label']):
      f.writelines(f'__label__{el} {et}\n')

"""
  takes the sample size, precision and recall as parameters and compute
  the F1 score. Print the 3 of these values. 
"""
def print_results(sample_size, precision, recall):
  precision = round(precision, 2)
  recall = round(recall, 2)
  f1 = 2 * ((precision * recall) / (precision + recall))
  print(sample_size, precision, recall, f1)
  
  return precision, recall, f1


In [ ]:
# sanity check
!head -n 10 sarcreddit4.txt

__label__0 i have a skylight over my bed, what do you think?
__label__0 ... and morgl the oracle
__label__0 the ground looks like play-doh.
__label__0 pollo tropical's cilantro garlic sauce
__label__1 damn clickbait.
__label__1 at least they only threatened to hit me in the head with a hammer once
__label__1 i think you've got it backwards friend.
__label__1 everyone except some us citizens is a moron then i guess
__label__1 hey man, that's my girl.
__label__1 today it's only 24 cents, but tomorrow, it's a ferrari...


In [11]:
# split data - be sure to reshuffle dataframe above for k-fold validation
l = df.shape[0]
tr = int(l * .8)
vl = round(l-tr)
print(tr)
print(vl)
assert tr+vl == l, "rounding error"

808660
202166


In [ ]:
! head -n {tr} sarcreddit5.txt > sarcred.train. # quickly convert files into expected format

In [ ]:
! tail -n {vl} sarcreddit5.txt > sarcred.valid

In [ ]:
! wc -l sarcred.valid

202166 sarcred.valid


In [ ]:
# define the model
model = fasttext.train_supervised('sarcred.train', lr=.1, dim=100, wordNgrams=2, loss='hs')

In [ ]:
# check performance
print_results(*model.test('sarcred.valid'))

202166 0.7 0.7 0.7


(0.7, 0.7, 0.7)

In [ ]:
model.test("sarcasm.valid", k=3) # output is test set size, precision (TP / TP + FP ), recall (TP / TP + FN)

#TP --> actual: sarcastic, predicted: sarcastic
#  FP --> actual: not sarcastic, predicted: sarcastic
#  FN --> actual: sarcastic, predicted: not sarcastic

In [ ]:
model.save_model("model.bin")

In [ ]:
# count the commment labels (pos or neg)
comments_labels = []
pos = 0
neg = 0
for i, comment in enumerate(sarc['comment']):
  if sarc['label'][i] == 0:
    neg += 1
  else:
    pos += 1
  comments_labels.append([sarc['label'][i], comment])

In [ ]:
# save data/ models here
savedir = "/content/gdrive/MyDrive/NLP_Final/data/txt"

In [ ]:
% cd /content/gdrive/MyDrive/NLP_Final/data/txt

/content/gdrive/.shortcut-targets-by-id/1gjC8j5numlBtyRH4ljSkR-XtOkBItgYp/NLP_Final/data/txt


In [ ]:
import fasttext
# define the model and train it, save as mod0
mod0 = fasttext.train_supervised(input="sarc0.train", lr=0.5, epoch=25, wordNgrams=2, dim=100, loss='hs') 

In [ ]:
mod1 = fasttext.train_supervised(input="sarc1.train", lr=0.5, epoch=25, wordNgrams=2, dim=100, loss='hs') 

In [ ]:
mod2 = fasttext.train_supervised(input="sarc2.train", lr=0.5, epoch=25, wordNgrams=2, dim=100, loss='hs') 

In [ ]:
mod3 = fasttext.train_supervised(input="sarc3.train", lr=0.5, epoch=25, wordNgrams=2, dim=100, loss='hs') 

In [ ]:
mod4 = fasttext.train_supervised(input="sarc4.train", lr=0.5, epoch=25, wordNgrams=2, dim=100, loss='hs') 

In [ ]:
mod5 = fasttext.train_supervised(input="sarc5.train", lr=0.1, epoch=25, wordNgrams=3, dim=50, loss='hs') 

In [ ]:
mod6 = fasttext.train_supervised(input="sarc6.train", lr=0.7, epoch=25, wordNgrams=3, dim=100, loss='hs') 

In [ ]:
mod0.save_model("mod0.bin")
mod1.save_model("mod1.bin")
mod2.save_model("mod2.bin")
mod3.save_model("mod3.bin")
mod4.save_model("mod4.bin")
mod5.save_model("mod5.bin")
mod6.save_model("mod6.bin")

In [ ]:
print("mod0 test")
mod0.test("sarc0.valid", k=5)

mod0 test


(101083, 0.5, 1.0)

In [ ]:
print("mod1 test")
mod1.test("sarc1.valid", k=5)

mod1 test


(101083, 0.5, 1.0)

In [ ]:
print("mod2 test")
mod2.test("sarc2.valid", k=5)

mod2 test


(101083, 0.5, 1.0)

In [ ]:
print("mod3 test")
mod3.test("sarc3.valid", k=5)

In [ ]:
print("mod4 test")
mod4.test("sarc4.valid", k=5)

mod4 test


(101083, 0.5, 1.0)

In [ ]:
print("mod5 test")
mod5.test("sarc5.valid", k=5)

mod5 test


(101083, 0.5, 1.0)

In [ ]:
mod0.predict("Nice perfume. did you shower in it?")

(('__label__1',), array([0.99815154]))

In [ ]:
mod1.predict("Nice perfume. did you shower in it?")


(('__label__1',), array([0.93067527]))

In [ ]:
mod2.predict("Nice perfume. did you shower in it?")

(('__label__1',), array([0.99996823]))

In [ ]:
mod3.predict("Nice perfume. did you shower in it?")

(('__label__1',), array([0.99977404]))

In [ ]:
mod6.predict("Nice perfume. did you shower in it?")

(('__label__0',), array([0.91405302]))

In [ ]:
help(fasttext.FastText)